In [3]:
#!wget https://huggingface.co/datasets/david4096/wildbio/blob/main/benchmark_lg_embed-distance.pkl
import pickle
with open('benchmark_lg_embed-distance.pkl', 'rb') as f:  # Open file in write-binary mode
    bdx = pickle.load(f) 

# this is the original dataset with added fields
# original question
bdx[168]['conversation'][0]
# original response
bdx[168]['conversation'][0]
# dictionary of LLM responses
bdx[168]['compare']
# dictionary of levenshtein distances
bdx[168]['response-ldistances']
# levenshtein similarity calculation to original response
bdx[168]['response-similarity']
# embedding of prompt
bdx[168]['prompt-embed']
#embedding of original response
bdx[168]['response-embed']
# dictionary of embeddings for responses
bdx[168]['embeddings']
# distance in embedding space to original response
bdx[168]['response-sdistances']


TypeError: list indices must be integers or slices, not str